In [25]:
import joblib
import numpy as np
import pandas as pd
from copy import deepcopy

# Carregar os artefatos salvos do main.py
modelo, X_test, explicacoes, features_usadas_lista, X_train, y_test, class_names = joblib.load('artefatos_para_teste_pi.pkl')

#modelo, X_test, explicacoes, features_usadas_lista, X_train = joblib.load('artefatos_para_teste_pi.pkl')
y_test = modelo.predict(X_test)  # Isso vai gerar as classes preditas
#class_names = ['Classe 0', 'Classe 1']


# Função para perturbar as features que não estão na explicação
def perturbar_instancia(instancia, features_a_manter, X_train):
    perturbada = instancia.copy()
    for f in instancia.index:
        if f not in features_a_manter:
            # Sorteia um valor realista da feature, com base nos dados de treino
            perturbada[f] = np.random.choice(X_train[f])
    return perturbada

# Função para validar a robustez das PI-explicações
def testar_robustez_pi(modelo, X_test, explicacoes, features_usadas_lista, X_train):
    total = 0
    mantidas = 0
    mudaram = []

    for idx, (inst_idx, instancia) in enumerate(X_test.iterrows()):
        features_usadas = features_usadas_lista[idx]
        pred_original = modelo.predict(instancia.to_frame().T)

        perturbada = perturbar_instancia(instancia, features_usadas, X_train)
        pred_perturbada = modelo.predict(instancia.to_frame().T)

        total += 1
        if pred_perturbada == pred_original:
            mantidas += 1
        else:
            mudaram.append(idx)
            print(f"[⚠️] Instância {idx}: Predição mudou após perturbação → {pred_original} ➜ {pred_perturbada}")

    print(f"\n✅ Robustez geral: {mantidas}/{total} ({(mantidas / total)*100:.2f}%) instâncias mantiveram a predição.")
    if mudaram:
        print(f"\nInstâncias que falharam no teste de robustez: {mudaram}")

# 🔥 CHAME A FUNÇÃO AQUI:
testar_robustez_pi(modelo, X_test, explicacoes, features_usadas_lista, X_train)



✅ Robustez geral: 20/20 (100.00%) instâncias mantiveram a predição.


In [26]:
def testar_robustez_visual(modelo, X_test, y_test, explicacoes, features_usadas_lista, X_train, class_names):
    for idx, (inst_idx, original) in enumerate(X_test.iterrows()):
        features_usadas = features_usadas_lista[idx]
        real_class = y_test[idx]
        pred_original = modelo.predict(original.to_frame().T)[0]

        perturbada = original.copy()
        diffs = {}
        for f in original.index:
            if f not in features_usadas:
                novo_valor = np.random.choice(X_train[f])
                if novo_valor != original[f]:
                    diffs[f] = (original[f], novo_valor)
                perturbada[f] = novo_valor

        pred_perturbada = modelo.predict(perturbada.to_frame().T)[0]
        status = "✅" if pred_perturbada == pred_original else "❌"

        print(f"\nInstância {idx} | Real: {class_names[real_class]} | Predita: {class_names[pred_original]} | Após perturbação: {class_names[pred_perturbada]} {status}")
        print(f"  → PI-explicação usou: {', '.join(features_usadas) if features_usadas else 'Nenhuma'}")
        perturbadas = [f for f in original.index if f not in features_usadas]
        print(f"  → Perturbadas: {', '.join(perturbadas)}")
        if diffs:
            print(f"  → Diferenças:")
            for f, (antes, depois) in diffs.items():
                print(f"     - {f}: {antes:.3f} → {depois:.3f}")
        else:
            print("  → Nenhuma diferença detectada")


In [27]:
testar_robustez_visual(modelo, X_test, y_test, explicacoes, features_usadas_lista, X_train, class_names)


Instância 0 | Real: virginica | Predita: virginica | Após perturbação: virginica ✅
  → PI-explicação usou: petal length (cm), sepal width (cm), sepal length (cm), petal width (cm)
  → Perturbadas: 
  → Nenhuma diferença detectada

Instância 1 | Real: virginica | Predita: virginica | Após perturbação: virginica ✅
  → PI-explicação usou: petal length (cm), petal width (cm), sepal length (cm), sepal width (cm)
  → Perturbadas: 
  → Nenhuma diferença detectada

Instância 2 | Real: virginica | Predita: virginica | Após perturbação: virginica ✅
  → PI-explicação usou: petal length (cm), petal width (cm), sepal width (cm), sepal length (cm)
  → Perturbadas: 
  → Nenhuma diferença detectada

Instância 3 | Real: versicolor | Predita: versicolor | Após perturbação: versicolor ✅
  → PI-explicação usou: petal length (cm), sepal length (cm), sepal width (cm), petal width (cm)
  → Perturbadas: 
  → Nenhuma diferença detectada

Instância 4 | Real: versicolor | Predita: versicolor | Após perturbação: